#### Использованные библиотеки

In [61]:
!pip install -r requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl
from torch import nn
from torch import optim
from torch.nn import functional as F

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from models import MLP, Transformer
from utils import mlp_params, gpt_params, g_transform
from datagen import SubLoader, TaskAugmentor
from pl_base import BestValidCallback

import logging
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)

from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

torch.set_float32_matmul_precision('medium')
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Перебор гиперпараметров

#### Setup

Первая важная вещь, которую делают в статье - это сравнение перцептрона и трансформера, как мета-моделей. Заявляется, что первый, сколько бы параметров у него не было, может только запоминать таски, а не обобщать. Трансформер наоборот должен уметь находить какие-то общие паттерны для всех тасок, благорадя которым он может работать хорошо на любом датасете. Демонстрирует это вот такая хитмапа:
<img src="https://cdn.discordapp.com/attachments/674191702906503199/1194322630292025394/image.png?ex=65afeea0&is=659d79a0&hm=4fab9e68c5d4c47d5f40599f0b341a9e0efa365bf35c14fd237191cb1e1c3705&">
Трансформер действительно что-то выучиывает, но на новых данных у него всё грустно (они впрочем этого не показывают вот здесь, зато я покажу), а трансформер - наоборот, это настоящее чудо

Это заявление мы будем проверять таким образом:
1. За модельки я возьму перцептрон с 2 скрытыми слоями и одним выходным, меняться у него будет только размерность скрытого слоя, и decoder-only трансформер, у которого меняю размер эмбеддингов. В качестве инпута будут выпрямленные 28x28x1 картинки из MNIST. Батч сайз 128, в статье 512, но у меня на таком ничего не получилось
2. В качестве датасета я возьму аугментированный $N_n$ раз MNIST. Сколько сэмплов брать, это вопрос очень интересный, в статье про это ни слова. Для MLP я возьму какое-то фиксированное число, а для трансформера по одному из каждой таски, этого будет достаточно, потому что в одной последовательности уже 100 примеров
   `train_loader` - MNIST, на котором тренируемся,
   `valid_loader` - другой мнист, с другими сэмплами, но с теми же тасками, на нём валидируемся,
   `test_loader` - тоже другой мнист, но теперь и с тасками другими, на нём тестим
3. Число тасок я буду перебирать по сетке с шагом в 2. В статье с шагом 1, но у меня нет столько времени. Для теста и валидации число равно $\log_2N_n$, чтобы не получилось слишком много. Сэмпл тоже 1, если трансформер, либо же 0.2, если MLP
4. Размер модели это тоже тонкая вещь. Для перцептрона понятно, что это размер хидден слоя. Для трансформера не так прозрачно. Он получается, как `hidden_size * n_heads * * 4`, и просто так его перебирать тяжко. Поэтому я решил забить и итерируюсь только по `hidden_size`
5. Для оптимизации перцептрона я не буду брать ничего хитрого, для трансформера хитрить придётся. В статье пишут, что на больших размерах тасок лосс выходит на плато, это чистая правда. Бороться с этим я буду по-своему - через оптимайзер и шедулер, кажется, что более-менее успешно

Остальное: параметры, функцию трейна, архитекруту моделек лучше посмотреть в коде внизу, либо в модулях. Ниже только код для обучения и получившиеся результаты

#### Интерпретация

Чтобы было поменьше копипасты, я засунул кусок с генерацией в отдельную функцию. Она делает 3 аугментора для каждого из 3 датасетов, 2 из них с одинковым сидом - это seen таски, третий с другим - это unseem, и также 3 лоадера

In [2]:
def augment_data(n_tasks, draw_sequence, batch_size, seed=69):

    # samples and augmentor
    train_samples = 1 if draw_sequence else min(60000*n_tasks, 1000000)
    test_samples = 1 if draw_sequence else min(60000*n_tasks, 100000)
    train_augmentor = TaskAugmentor(
        n_tasks, draw_sequence=draw_sequence, random_state=seed
    )
    seen_augmentor = TaskAugmentor(
        int(np.log2(n_tasks)), draw_sequence=draw_sequence, random_state=seed
    )
    unseen_augmentor = TaskAugmentor(
        int(np.log2(n_tasks)), draw_sequence=draw_sequence, random_state=seed+1
    )

    # loaders
    train_loader = DataLoader(
        train_augmentor.transform(mnist_train, train_samples),
        batch_size=batch_size, shuffle=True
    )
    valid_loader = DataLoader(
        seen_augmentor.transform(mnist_test, test_samples),
        batch_size=batch_size, shuffle=False
    )
    test_loader = DataLoader(
        unseen_augmentor.transform(mnist_test, test_samples),
        batch_size=batch_size, shuffle=False
    )
    return train_loader, valid_loader, test_loader

MNIST, из которого я буду сэмплить, оптимайзеры, трансформы, это всё одинаковое вне зависимости от конфигурации, поэтому их я собираю отдельно в `utils.py`

In [2]:
mnist_train = SubLoader(MNIST('./datasets', train=True, transform=g_transform)) 
mnist_test = SubLoader(MNIST('./datasets', train=True, transform=g_transform))

Наконец остаётся только запустить цикл и идти пить чай. По-хорошему это нужно делать для нескольких сидов, но на это нужно время. Результаты на всякий случай засылались в том числе и на [wandb](https://wandb.ai/lerostre/gpicl/runs/i57grjqb/overview?workspace=user-lerostre), хотя там не очень информативно

Функция ниже консервирует параметры, которые менять всё же не следует. Что поменять можно:
название модели, но в статье это делается только для двух, если надо что-то ещё, можно засунуть прямо в код, батч сайз, логгирование в wandb, таски для перебора, hidden size для перебора, суффикс для выходного файла, сид (у меня опять один, мне лень), а также прочие параметры для тренера, если это нужно

In [ ]:
def train_log_loop(
    model_name="gpt",
    batch_size=128,
    logger=False,
    n_tasks_list=2**np.arange(0, 19, 2),
    hid_size_list=2**np.arange(0, 10),
    output_file_suffix="",
    random_state=0,
    **trainer_arguments
):
    # each model has its own metric df
    log = pd.DataFrame(columns=["model", "dataset", "n_tasks", "accuracy", "hid_size"])
    
    for n_tasks in n_tasks_list:
        #each task has its own loader
        loaders = augment_data(
            n_tasks, draw_sequence=True,
            batch_size=batch_size, seed=random_state
        )
        train_loader, valid_loader, test_loader = loaders
        
        for hid_size in hid_size_list:
            # different models are differently initialised
            if model_name == "gpt":
                config = GPT2Config(
                    num_labels=10, hidden_size=int(hid_size), n_inner=int(4*hid_size),
                    n_layer=4, n_head=1, n_positions=100,
                )
                model = GPT(config=config, **params))
                # number of epochs can be changed if necessary but best not to
                training_config = {"max_steps": 100000}
            elif model_name == "mlp":
                model = MLP(hidden_size=hid_size, **params)
                training_config = {"max_epochs": 10}
                
            # init wandb run for given model
            wandb_logger = None
            if logger:
                # config is a bit clunky to track
                name = f"{model_name}_{n_tasks}_{hid_size}"
                # for param, param_value in model_kwargs.items():
                #     name += f"__{param}_{param_value}"
                # wandb_config = {
                #     "name": model_name,
                # }
                # wandb_config.update(trainer_arguments)
                # wandb_config.update(training_arguments)
                wandb.init(
                    project="gpicl", name=name, tags=[model_name],
                    # config=wandb_config
                )
                wandb_logger = WandbLogger(log_model=False)
        
            # train and store
            trainer = pl.Trainer(
                precision="16",
                accelerator="gpu",
                logger=wandb_logger,
                enable_progress_bar=False,
                enable_model_summary=False,
                **trainer_arguments, **training_config
            )
            trainer.fit(model, train_loader, valid_loader)
            logs = trainer.callback_metrics
            logs.update(trainer.test(model, test_loader, verbose=False)[0])
            logs["valid_accuracy"] = model.best_valid_acc
            
            clear_output(True)

            # fill df with metrics
            for name, metric in train_results.items():
                if "accuracy" in name:
                    dataset = name.split("_")[0]
                    if isinstance(metric, torch.Tensor):
                        metric = metric.item()
                    log.loc[log.shape[0]] = (model_name, dataset, n_tasks, metric, hid_size)
                    log.to_csv(f"experiments/{model_name}_log{output_file_suffix}.csv", index=0)

In [ ]:
for model in ["mlp", "gpt"]:
    train_log_loop(model)

Generating tasks:   0%|          | 0/262144 [00:00<?, ?it/s]

Если всё сделано правильно, то в папке должны были появиться 2 лога для перцептрона и трансформера. Осталось их считать и построить очень красивый график

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly
from IPython.display import display, HTML
from plotly.subplots import make_subplots
from itertools import product

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

def df_to_plotly(df):
    return {'z': df.accuracy.tolist(),
            'x': df.hid_size.astype(str).tolist(),
            'y': df.n_tasks.astype(str).tolist()}

titles = [
    f'{model}, {seen} MNIST' for model, seen
    in product(['MLP', 'Transformer'], ['seen', 'unseen'])
]
seen_mapper = {"valid": "seen", "test": "unseen"}
fig = make_subplots(
    rows=2, cols=2,
    shared_yaxes=True,
    subplot_titles=tuple(titles),
    vertical_spacing = 0.15
)

log_cols = ['model', 'dataset', 'n_tasks', 'hid_size', 'accuracy']
for i, model in enumerate(['MLP', 'Transformer']):
    for j, seen in enumerate(["seen", "unseen"]):
        sublog = pd.read_csv(f"experiments/{model.lower()}_log.csv")
        sublog = sublog.sort_values(['n_tasks', 'hid_size'])
        sublog["dataset"] = sublog.dataset.map(seen_mapper)
        sublog = sublog[sublog.dataset == seen]
        plotly_df = df_to_plotly(sublog)
        fig.add_trace(go.Heatmap(
            **plotly_df, coloraxis="coloraxis", zmin=0, zmax=1,
        ), row=i+1, col=j+1)
        
for i in [1, 3]:
    fig["layout"][f"yaxis{i}"].update(
        title_text='number of tasks', tickmode='array',
        tickvals = np.sort(np.unique(plotly_df["y"]).astype(int)),
        ticktext = [f"$2^{{{i}}}$" for i in range(0, 17, 2)]
    )
fig.update_xaxes(
    title_text='hidden size', showgrid=False,
    tickvals = np.sort(np.unique(plotly_df["x"]).astype(int)),
    ticktext = [f"${i}$" for i in np.sort(np.unique(plotly_df["x"]).astype(int))]
)
fig.update_yaxes(showgrid=False)
fig.update_layout(
    height=700, width=700,
    coloraxis={"colorbar":dict(title="Accuracy"), 'colorscale':'Brwnyl'},
    plot_bgcolor="#E6D7BD", title="Performance on seen and unseen tasks"
)
fig

<img src="https://cdn.discordapp.com/attachments/674191702906503199/1194467621245030450/newplot_3.png?ex=65b075a9&is=659e00a9&hm=6719ed2f15771a23e129217786f360e03372fd842fadaca0f0834dd4b64d8be4&">

Сложно сказать, получилось ли то же, потому что в статье нет чётких цифр. Можно видеть, что эта штука реально работает, почему=то, картинка похожа. Другое дело, что качество на самом-то деле не очень большое - в районе 0.5 аккураси, но тот факт, что но вообще научилось обобщать, это конечно поразительно. На графике если что есть наны, всё-таки я не все параметры перебрал, да и тех, что есть чуть меньше, но всё сочное должно тут быть